In [1]:
import numpy as np
import scipy.io.wavfile
from scipy.fftpack import dct
from scipy.signal import savgol_filter
from PyEMD import EMD


def first_step(sample_rate,signal):
    
    ##### Pre-Emphasis
    #set filter coefficient for pre-emphasis  filter coefficient (α) 
    pre_emphasis = 0.97 #typical value need to be set                                                 **
    
    emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
    
    ##### sav_gol filter                                                                               **
    #link:- https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.signal.savgol_filter.html
    np.set_printoptions(precision=2)  # For compact display.
    after_savgol_signal = savgol_filter(emphasized_signal, 5, 2, mode = 'nearest')
    # 5 is the window length and 2 is the polynomial degree 
    
    
    ##### emd
    #link :- https://github.com/laszukdawid/PyEMD
    emd = EMD()        #basic emd
    imfs = emd(after_savgol_signal)
    
    
    
    for i in imfs:
        #### Framing
        #Typical frame sizes in speech processing range from 20 ms to 40 ms 
        #with 50% (+/-10%) overlap between consecutive frames. 
        #Popular settings are 25 ms for the frame size,  and a 10 ms stride (15 ms overlap), 
        #link :- https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html
        frame_size = 0.025
        frame_stride = 0.01
        frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
        signal_length = len(i)
        frame_length = int(round(frame_length))
        frame_step = int(round(frame_step))
        num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

        pad_signal_length = num_frames * frame_step + frame_length
        z = numpy.zeros((pad_signal_length - signal_length))
        pad_signal = numpy.append(i, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

        indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
        frames = pad_signal[indices.astype(numpy.int32, copy=False)]
        
        
        
        ######
        
        
    
    